In [1]:
import pandas as pd
import numpy as np

# 運用到前一日收盤價事件
(一) 新增或刪除採樣股票生效日。 : 
調整市值＝前一日收盤價 × 發行股數   

(三) 員工酬勞增資股或新股權利證書上市日。 : 
調整市值＝除權日前一日收盤×員工分紅(仟股) 

(六) 公司依法註銷股份辦理減資經本公司公告後之除權交易日或次月第三個交易日，並以較先者為準。
調整市值 = 除權日前一日收盤×庫藏股註銷(仟股) 

(八) 公司合併後增資股或新股權利證書上市日。
調整市值 = 除權日前一日收盤×合併(仟股)

(九) 轉換公司債轉換的債券換股權證換發為普通股的上市日。
調整市值 = 除權日前一日收盤×證券轉換(仟股)

(一二) 為海外存託憑證而發行的新股上市日。
調整市值 = 除權日前一日收盤× GDR比率% ×現金增資(仟股)

(一三) 可轉換特別股轉換為普通股的上市日。
調整市值 =除權日前一日收盤×證券轉換(仟股)

# 運用到其他日期事件
(二) 現金增資認購普通股的除權交易日。:
調整市值＝現金增資認購價 × 現金增資股數

# 暫時不討論事件
(三) 員工酬勞增資股或新股權利證書上市日。 : 
調整市值＝除權日前一日收盤×員工分紅(仟股) 
新股權利證書Case2: 
現金增資發行新股參與發行海外存託憑證 - 發行GDR 且員工認股
現金增資員工認股
(四) 特別股無償配發普通股除權交易日。 : 
(五) 上市公司持有未辦理減資註銷庫藏股除權交易日。    
(七) 收到現金增資募集失敗之通知後，次月第三個交易日將發行股數復原。
(一〇)上市公司發行之轉換公司債直接換發為普通股或附認股權有價證券(附認股權公司債及附. 認股權特別股)認購而發行之普通股，俟其除權交易日或其辦理資本額變更登記經本公司公告後次月第三個交易日。
(一一) 股東放棄認購而採公開承銷之現金增資股票或股款繳納憑證上市日。


# 一、 匯入所需資料、事件資料以及日股價資料(取得除權前一日收盤價)

In [2]:
df_Event = pd.read_csv('./基值相關/Event.txt',engine='python',sep='\t')
df_Event['現金認購價（元）'] = df_Event['現金認購價（元）'].str.replace(' ','').replace('-',np.NaN).astype('float')

In [3]:
df_D = pd.read_csv('./Data/Daily_Data.txt',engine='python',sep='\t')
df_D = df_D.sort_values(['證券代碼','年月日']).reset_index(drop=True)
df_D['除權前一日收盤價'] = df_D.groupby('證券代碼')['收盤價'].shift(1)


# 二、 整理所有事件，輸出為各式件資料集

# (一) 新增或刪除採樣股票生效日。 : 

In [4]:
df_final = pd.read_csv('df_final.csv',engine='python',encoding='utf_8_sig')
df_final = df_final[['Data_End_Date', 'Review_Date', 'Active_Date', 'Code', 'New_Remain_Del','SmartBeta']]

In [5]:
temp = pd.merge(left=df_final,right=df_D[['證券代碼','簡稱','年月日','除權前一日收盤價','流通在外股數']],left_on=['Code','Active_Date'],right_on=['證券代碼','年月日'],how='left')
temp = temp.dropna().reset_index(drop=True)

In [6]:
# 取出每次審核日新加入的個股
temp = temp[temp['New_Remain_Del']==0].reset_index(drop=True)
df_New = pd.DataFrame()
df_New['除權日'] = temp['年月日']
df_New['公司'] = temp['證券代碼']
df_New['簡稱'] = temp['簡稱']
df_New['事件名稱'] = ['New']*len(df_New)
df_New['調整市值'] =  temp['除權前一日收盤價'] *  temp['流通在外股數']

In [7]:
temp = pd.merge(left=df_final,right=df_D[['證券代碼','簡稱','年月日','除權前一日收盤價','流通在外股數']],left_on=['Code','Active_Date'],right_on=['證券代碼','年月日'],how='left')


In [8]:
# 取出每次審核日刪除的個股
temp = temp[temp['New_Remain_Del']==2].reset_index(drop=True)
df_Del = pd.DataFrame()
df_Del['除權日'] = temp['年月日']
df_Del['公司'] = temp['證券代碼']
df_Del['簡稱'] = temp['簡稱']
df_Del['事件名稱'] = ['Del']*len(df_Del)
df_Del['調整市值'] =  temp['除權前一日收盤價'] *  temp['流通在外股數']

In [9]:
df_Del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 5 columns):
除權日     179 non-null int64
公司      179 non-null int64
簡稱      179 non-null object
事件名稱    179 non-null object
調整市值    179 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.1+ KB


# (二) 現金增資認購普通股的除權交易日。

In [10]:
temp = df_Event[df_Event['GDR 比率 %']==0]  #由於新增海外存託憑證也是使用到現金增資(仟股)欄位，故取

df_SEO = pd.DataFrame()
df_SEO['除權日'] = temp['除權日']
df_SEO['公司'] = temp['公司']
df_SEO['簡稱'] = temp['簡稱']
df_SEO['事件名稱'] = ['SEO']*len(df_SEO)
df_SEO['調整市值'] =  temp['現金認購價（元）'] *  temp['現金增資(仟股)']

In [11]:
df_SEO =df_SEO[ df_SEO['調整市值'] != 0].reset_index(drop=True)

In [12]:
# 檢查有無同一公司在同天增資兩次
len(df_SEO)==len(df_SEO.groupby(['除權日','公司'])['調整市值'].sum())

True

# (三) 員工酬勞增資股或新股權利證書上市日。 : 

In [13]:
df_Event['公司'] = df_Event['公司'].str.replace(' ','').astype('str')
df_D['證券代碼']= df_D['證券代碼'].astype('str')
temp = pd.merge(left=df_Event,right=df_D[['證券代碼','年月日','除權前一日收盤價']],left_on=['公司','除權日'],right_on=['證券代碼','年月日'],how='left')
temp = temp.dropna().reset_index(drop=True)

In [14]:
df_Staff = pd.DataFrame()
df_Staff['除權日'] = temp['除權日']
df_Staff['公司'] = temp['公司']
df_Staff['簡稱'] = temp['簡稱']
df_Staff['事件名稱'] = ['Staff']*len(df_Staff)
df_Staff['調整市值'] =  temp['除權前一日收盤價'] *  temp['員工分紅(仟股)']

In [15]:
df_Staff =df_Staff[ df_Staff['調整市值'] != 0].reset_index(drop=True)

In [16]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_Staff)==len(df_Staff.groupby(['除權日','公司'])['調整市值'].sum())

True

# (六) 公司依法註銷股份辦理減資經本公司公告後之除權交易日或次月第三個交易日，並以較先者為準。

In [17]:
df_Treasury = pd.DataFrame()
df_Treasury['除權日'] = temp['除權日']
df_Treasury['公司'] = temp['公司']
df_Treasury['簡稱'] = temp['簡稱']
df_Treasury['事件名稱'] = ['Treasury']*len(df_Treasury)
df_Treasury['調整市值'] =  temp['除權前一日收盤價'] *  temp['庫藏股註銷(仟股)']

In [18]:
df_Treasury =df_Treasury[ df_Treasury['調整市值'] != 0].reset_index(drop=True)

In [19]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_Treasury)==len(df_Treasury.groupby(['除權日','公司'])['調整市值'].sum())

True

# (八) 公司合併後增資股或新股權利證書上市日。

In [20]:
df_Merge = pd.DataFrame()
df_Merge['除權日'] = temp['除權日']
df_Merge['公司'] = temp['公司']
df_Merge['簡稱'] = temp['簡稱']
df_Merge['事件名稱'] = ['Merge']*len(df_Merge)
df_Merge['調整市值'] =  temp['除權前一日收盤價'] *  temp['合併(仟股)']

In [21]:
df_Merge =df_Merge[ df_Merge['調整市值'] != 0].reset_index(drop=True)

In [22]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_Merge)==len(df_Merge.groupby(['除權日','公司'])['調整市值'].sum())

True

# (九) 轉換公司債轉換的債券換股權證換發為普通股的上市日。

In [23]:
temp_CB=temp[[  ('債' in x)or('CB' in x)or('cb'in x) for x in temp['其他-說明']]]

In [24]:
df_CB = pd.DataFrame()
df_CB['除權日'] = temp_CB['除權日']
df_CB['公司'] = temp_CB['公司']
df_CB['簡稱'] = temp_CB['簡稱']
df_CB['事件名稱'] = ['CB']*len(df_CB)
df_CB['調整市值'] =  temp_CB['除權前一日收盤價'] *  temp_CB['証券轉換(仟股)']

In [25]:
df_CB =df_CB[ df_CB['調整市值'] != 0].reset_index(drop=True)

In [26]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_CB)==len(df_CB.groupby(['除權日','公司'])['調整市值'].sum())

True

# (一二) 為海外存託憑證而發行的新股上市日。

In [27]:
temp_GDR = temp[temp['GDR 比率 %']!=0]

In [28]:
df_GDR = pd.DataFrame()
df_GDR['除權日'] = temp_GDR['除權日']
df_GDR['公司'] = temp_GDR['公司']
df_GDR['簡稱'] = temp_GDR['簡稱']
df_GDR['事件名稱'] = ['GDR']*len(df_GDR)
df_GDR['調整市值'] =  temp_GDR['除權前一日收盤價'] *  temp_GDR['現金增資(仟股)']

In [29]:
df_GDR =df_GDR[ df_GDR['調整市值'] != 0].reset_index(drop=True)

In [30]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_GDR)==len(df_GDR.groupby(['除權日','公司'])['調整市值'].sum())

True

# (一三) 可轉換特別股轉換為普通股的上市日。

In [31]:
temp_CP=temp[[  ('甲特' in x)or('乙特' in x)or('丙特' in x) for x in temp['其他-說明']]]

In [32]:
df_CP = pd.DataFrame()
df_CP['除權日'] = temp_CP['除權日']
df_CP['公司'] = temp_CP['公司']
df_CP['簡稱'] = temp_CP['簡稱']
df_CP['事件名稱'] = ['CP']*len(df_CP)
df_CP['調整市值'] =  temp_CP['除權前一日收盤價'] *  temp_CP['証券轉換(仟股)']

In [33]:
df_CP =df_CP[ df_CP['調整市值'] != 0].reset_index(drop=True)

In [34]:
# 檢查有無同一公司在同天增資兩次 True 表示沒有
len(df_CP)==len(df_CP.groupby(['除權日','公司'])['調整市值'].sum())

True

# 三、合併個事件資料，並檢查單一個股同天是否發生多事件

In [35]:
dfs = [df_New,df_Del,df_CB,df_CP,df_GDR,df_Merge,df_SEO]
df_final=pd.concat(dfs).reset_index(drop=True)
df_final['調整市值'] = df_final['調整市值']*1000

In [36]:
print(len(df_final[pd.isnull(df_final['調整市值'])==True]))
print((pd.isnull(df_Event['現金認購價（元）'])*1).sum())

15
15


In [37]:

df_final['公司'] = df_final['公司'].astype('str')
df_final=df_final.dropna().reset_index(drop=True)

In [38]:
print(len(df_final)==len(df_final.drop_duplicates(['除權日','公司'])) )#檢查有無同一天發生兩件事 False表示有一公司同天發生兩件事
df_final[df_final.duplicated(['除權日','公司'],keep=False)]

True


,除權日,公司,簡稱,事件名稱,調整市值


In [40]:
df_final.to_csv('df_final_event.csv',index=False,encoding='utf_8_sig')